In [2]:
import sys
sys.path.append('../..')
import os
from tqdm import tqdm
from config import client
from pymongo import MongoClient
import pandas as pd

import matplotlib.pyplot as plt
import math
import numpy as np
#import json 
#from pandas.io.json import json_normalize #package for flattening json in pandas df

In [3]:
#client = MongoClient(client)
db = client['osu_random_db']

## Import data

In [4]:
sample_id = list(db.week_9_sample_users.find( {}, {"_id":0, "users": 1}))[0]["users"]

In [5]:
user_info = {}
for user in sample_id:
    user_info[user] = list(db.osu_scores_high.find( {"user_id": user, "enabled_mods": 0}, {"_id": 0, "beatmap_id": 1, "count50": 1, "count100": 1, "count300": 1, "countmiss": 1, "countgeki": 1, "countkatu": 1, "perfect": 1}))

In [ ]:
user_info_with_beatmap_features = user_info
df = pd.DataFrame()
for user in sample_id:
    for bm in user_info_with_beatmap_features[user]:
        bm.update(list(db.osu_beatmaps.find( {"_id": bm["beatmap_id"]}, {"_id": 0, "countNormal": 1, "countSlider": 1, "countSpinner": 1, "countTotal": 1, "diff_approach": 1, "diff_drain": 1, "diff_overall": 1, "diff_size": 1, "diffcultyrating": 1, "hit_length": 1}))[0])
        bm.update(list(db.osu_scores_high.find( {"user_id": user, "beatmap_id": bm["beatmap_id"], "enabled_mods": 0}, {"_id": 0, "mlpp.est_user_pp": 1}))[0]["mlpp"])
        df.append(pd.DataFrame(user_info_with_beatmap_features[user]))

In [ ]:
user_info_with_beatmap_features = user_info
df = pd.DataFrame()
for user in sample_id:
    for bm in user_info_with_beatmap_features[user]:
        bm.update(list(db.osu_beatmaps.find( {"_id": bm["beatmap_id"]}, {"_id": 0, "countNormal": 1, "countSlider": 1, "countSpinner": 1, "countTotal": 1, "diff_approach": 1, "diff_drain": 1, "diff_overall": 1, "diff_size": 1, "diffcultyrating": 1, "hit_length": 1}))[0])
        bm.update(list(db.osu_scores_high.find( {"user_id": user, "beatmap_id": bm["beatmap_id"], "enabled_mods": 0}, {"_id": 0, "mlpp.est_user_pp": 1}))[0]["mlpp"])
    user_df = pd.DataFrame(user_info_with_beatmap_features[user])
    df = df.append(user_df)

In [44]:
df.to_pickle("xgboostDf") 

<bound method NDFrame.tail of      beatmap_id  count50  count100  count300  countmiss  countgeki  countkatu  \
0          2037        2        76       133          5          4         20   
1          2409        6        24        70          5         14         11   
2          2607        2         7       185          1         36          5   
3          3670        2        19       230         11         63         11   
4          3851        5        41        87          9         38         19   
..          ...      ...       ...       ...        ...        ...        ...   
117      769312        2        19       194          3         56         11   
118     1054270        1        12        44          4          6          6   
119     1091309        3        68       250         10         18         33   
120     1093529        8        84       207         14         27         41   
121     1095534       18       179       687         26        102         97  

In [34]:
user_info_with_beatmap_features[firstUser]

[{'beatmap_id': 2037,
  'count50': 2,
  'count100': 76,
  'count300': 133,
  'countmiss': 5,
  'countgeki': 4,
  'countkatu': 20,
  'perfect': 0,
  'countNormal': 155,
  'countSlider': 60,
  'countSpinner': 1,
  'countTotal': 278,
  'diff_approach': 5.0,
  'diff_drain': 4.0,
  'diff_overall': 5.0,
  'diff_size': 5.0,
  'hit_length': 155,
  'est_user_pp': 114.7568405839929},
 {'beatmap_id': 2409,
  'count50': 6,
  'count100': 24,
  'count300': 70,
  'countmiss': 5,
  'countgeki': 14,
  'countkatu': 11,
  'perfect': 0,
  'countNormal': 79,
  'countSlider': 24,
  'countSpinner': 2,
  'countTotal': 133,
  'diff_approach': 5.0,
  'diff_drain': 5.0,
  'diff_overall': 5.0,
  'diff_size': 5.0,
  'hit_length': 64,
  'est_user_pp': 132.59461103588794},
 {'beatmap_id': 2607,
  'count50': 2,
  'count100': 7,
  'count300': 185,
  'countmiss': 1,
  'countgeki': 36,
  'countkatu': 5,
  'perfect': 0,
  'countNormal': 158,
  'countSlider': 37,
  'countSpinner': 0,
  'countTotal': 232,
  'diff_approach'

In [37]:
df = df.append(pd.DataFrame(user_info_with_beatmap_features[user]))

In [38]:
second =sample_id[1]
user_info_with_beatmap_features[second]
df.append(pd.DataFrame(user_info_with_beatmap_features[second]))

,beatmap_id,count50,count100,count300,countmiss,countgeki,countkatu,perfect,countNormal,countSlider,countSpinner,countTotal,diff_approach,diff_drain,diff_overall,diff_size,hit_length,est_user_pp
0,275,3,31,231,8,90,26,0,199,0,0,273,5.0,5.0,5.0,5.0,222,61.699066
1,335,0,12,84,7,19,7,0,73,0,0,103,4.0,3.0,4.0,5.0,86,12.092719
2,336,8,19,104,7,13,11,0,106,0,0,138,5.0,3.0,5.0,5.0,85,60.641402
3,1087,0,10,107,4,23,5,0,82,37,2,162,5.0,5.0,5.0,4.0,99,88.152249
4,1532,0,13,110,8,13,11,0,108,22,1,155,3.0,4.0,3.0,5.0,101,49.889647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,769312,2,19,194,3,56,11,0,61,152,5,380,4.5,3.5,4.0,3.5,149,169.307610
118,1054270,1,12,44,4,6,6,0,30,30,1,93,4.5,3.0,4.0,3.6,36,169.839705
119,1091309,3,68,250,10,18,33,0,163,167,1,500,5.0,2.0,4.5,3.5,197,168.736622
120,1093529,8,84,207,14,27,41,0,106,205,2,522,8.0,5.0,6.0,3.8,107,193.075767
